Zero-shot (instruction-only)

In [ ]:
from openai import OpenAI
import os

client = OpenAI(base_url=os.environ["NIM_BASE_URL"], api_key=os.getenv("NIM_API_KEY","nvapi-..."))

def zero_shot(prompt: str, user_input: str):
    return client.chat.completions.create(
        model=os.environ["NIM_MODEL"],
        messages=[
            {"role":"system","content": prompt},
            {"role":"user","content": user_input},
        ],
        temperature=0.2,
        top_p=0.9,
        max_tokens=200
    ).choices[0].message.content


One-shot / Few-shot (demonstrations as message pairs)

In [ ]:
def few_shot(system_prompt: str, demos: list[tuple[str,str]], user_input: str):
    messages = [{"role":"system","content": system_prompt}]
    for x, y in demos:
        messages += [
            {"role":"user","content": x},
            {"role":"assistant","content": y},
        ]
    messages.append({"role":"user","content": user_input})

    return client.chat.completions.create(
        model=os.environ["NIM_MODEL"],
        messages=messages,
        temperature=0.2,
        top_p=0.9,
        max_tokens=200
    ).choices[0].message.content


Example demos:

In [ ]:
system_prompt = "Classify priority as P0/P1/P2/P3. Output only the label."

demos = [
  ("Service down for all users. No workaround.", "P0"),
  ("Intermittent errors for 10% users. Workaround exists.", "P1"),
  ("Minor UI bug. No impact on core flow.", "P3"),
]

print(few_shot(system_prompt, demos, "Checkout failures in one region, workaround: manual retry."))
